In [7]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL External Data-set")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [8]:
# Load income by SA2 df
income_by_sa2 = spark.read.parquet('../../data/tables/income_by_sa2.parquet')

In [9]:
transactions = spark.read.parquet('../../data/curated/process_data.parquet')

In [10]:
#income_by_sa2.select('SA2').distinct().count()
#income_by_sa2.columns

2288

Only getting 2288 distinct SA2 district's from the ABS data set -> some must be missing?

In [11]:
# read in postcode linking file
postcodes = spark.read.options(header=True).csv('../../data/tables/external_datasets/postcode_data.csv')

In [13]:
postcodes_feature_subset = ['postcode', 'SA2_MAINCODE_2016', 'locality', 'lat', 'long']
postcodes = postcodes.select(*postcodes_feature_subset)

In [14]:
postcodes.limit(3)

postcode,SA2_MAINCODE_2016,locality,lat,long
0200,801051049,ANU,-35.2777,149.119
0200,801051049,Australian Nation...,-35.2777,149.1189
0800,701011002,DARWIN,-12.458684,130.83668


Make a subset just for linking

In [15]:
link_set =  ['postcode', 'SA2_MAINCODE_2016']
link = postcodes.select(*link_set)
link.limit(3)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002


In [16]:
# descriptive statistics
print("Distinct postcodes: ", link.select('postcode').distinct().count())
print("Distinct SA2 regions: ", link.select('SA2_MAINCODE_2016').distinct().count())

Distinct postcodes:  3167


Distinct SA2 regions:  2222


Again -> this is less than the officially reported number of ~ 2310, might have to look into using geopandas and closest centroids...

#### Take the income dataset; match sa2 to postcodes; aggregate across postcodes for final set

In [17]:
link.printSchema()

root
 |-- postcode: string (nullable = true)
 |-- SA2_MAINCODE_2016: string (nullable = true)



In [18]:
income_by_sa2.printSchema()

root
 |-- SA2: string (nullable = true)
 |-- SA2_NAME: string (nullable = true)
 |-- persons_earners_2014-15: string (nullable = true)
 |-- persons_earners_2015-16: string (nullable = true)
 |-- persons_earners_2016-17: string (nullable = true)
 |-- persons_earners_2017-18: string (nullable = true)
 |-- persons_earners_2018-19: string (nullable = true)
 |-- med_age_earners_2014-15: string (nullable = true)
 |-- med_age_earners_2015-16: string (nullable = true)
 |-- med_age_earners_2016-17: string (nullable = true)
 |-- med_age_earners_2017-18: string (nullable = true)
 |-- med_age_earners_2018-19: string (nullable = true)
 |-- sum_earnings_2014-15: string (nullable = true)
 |-- sum_earnings_2015-16: string (nullable = true)
 |-- sum_earnings_2016-17: string (nullable = true)
 |-- sum_earnings_2017-18: string (nullable = true)
 |-- sum_earnings_2018-19: string (nullable = true)
 |-- median_earnings_2014-15: string (nullable = true)
 |-- median_earnings_2015-16: string (nullable = true)


In [19]:
income_by_sa2.limit(3)

22/09/08 22:26:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2,SA2_NAME,persons_earners_2014-15,persons_earners_2015-16,persons_earners_2016-17,persons_earners_2017-18,persons_earners_2018-19,med_age_earners_2014-15,med_age_earners_2015-16,med_age_earners_2016-17,med_age_earners_2017-18,med_age_earners_2018-19,sum_earnings_2014-15,sum_earnings_2015-16,sum_earnings_2016-17,sum_earnings_2017-18,sum_earnings_2018-19,median_earnings_2014-15,median_earnings_2015-16,median_earnings_2016-17,median_earnings_2017-18,median_earnings_2018-19,mean_earnings_2014-15,mean_earnings_2015-16,mean_earnings_2016-17,mean_earnings_2017-18,mean_earnings_2018-19
101021007,Braidwood,2133,2153,2262,2315,2361,50,50,50,51,51,101830606,109961530,115564780,119439615,120763285,38093,39716,41288,42003,41593,47741,51074,51090,51594,51149
101021008,Karabar,4866,4937,4988,5059,5100,43,42,42,42,42,289762288,301618607,311923113,322715297,338308979,54942,55870,57880,59295,61777,59548,61093,62535,63790,66335
101021009,Queanbeyan,6194,6419,6486,6595,6697,40,39,39,39,39,367366063,390044004,395374979,414579067,441160946,54716,54999,55376,57848,60119,59310,60764,60958,62863,65874


In [20]:
link.limit(3)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002


In [31]:
link = link.distinct()
link = link.withColumnRenamed('SA2_MAINCODE_2016', 'sa2_code')
link.orderBy('postcode').limit(7)

postcode,sa2_code
0200,801051049
0800,701011002
0801,701011002
0804,701011007
0810,701021021
0810,701021023
0810,701021029


In [56]:
inc_joined = link.join(income_by_sa2, link.sa2_code == income_by_sa2.SA2, "left").na.drop().distinct()
#inc_joined = link.join(income_by_sa2, link.SA2_MAINCODE_2016 == income_by_sa2.SA2, "left").distinct()
print(inc_joined.count())
inc_joined.select(['postcode', 'sa2_code']).orderBy('postcode').limit(7)

# end up with 5588 rows
# therefore there a lot of postcodes with SA2's > 1 || SA2's with postcodes > 1

5487


postcode,sa2_code
0200,801051049
0800,701011002
0801,701011002
0804,701011007
0810,701021021
0810,701021023
0810,701021029


In [33]:
#inc_joined.select('SA2').distinct().count()
#inc_joined.select('postcode').distinct().count()

2218

In [17]:
from pyspark.sql import functions as F

In [18]:
#inc_cols = inc_joined.columns

In [19]:
# num of distinct postcodes
inc_joined.select('postcode').distinct().count()

3167

In [34]:
agg_by_postcode_income = inc_joined \
    .groupBy('postcode') \
    .agg(
    F.sum('persons_earners_2018-19').alias('persons_earners_2018-19_sum'),
    F.mean('mean_earnings_2018-19').alias('mean_earnings_2018-19_avg'),
    F.sum('sum_earnings_2018-19').alias('sum_earnings_2018-19_sum'),
    F.mean('median_earnings_2018-19').alias('median_earnings_2018-19_avg'),
    F.mean('med_age_earners_2018-19').alias('med_age_earners_2018-19_avg')
    #F.percentile_approx('median_earnings_2018-19', 0.5).alias('median_earnings_2018-19_med'),
    #F.percentile_approx('med_age_earners_2018-19', 0.5).alias('med_age_earners_2018-19_med')
) \
    .orderBy('postcode')

In [35]:
agg_by_postcode_income.show()

+--------+---------------------------+-------------------------+------------------------+---------------------------+---------------------------+
|postcode|persons_earners_2018-19_sum|mean_earnings_2018-19_avg|sum_earnings_2018-19_sum|median_earnings_2018-19_avg|med_age_earners_2018-19_avg|
+--------+---------------------------+-------------------------+------------------------+---------------------------+---------------------------+
|    0200|                      552.0|                  19479.0|             1.0752338E7|                    10433.0|                       23.0|
|    0800|                     5632.0|                  74682.0|            4.20609031E8|                    57789.0|                       33.0|
|    0801|                     5632.0|                  74682.0|            4.20609031E8|                    57789.0|                       33.0|
|    0804|                     1810.0|                  88303.0|            1.59828824E8|                    71724.0|       

In [37]:
#import sys
#sys.path.append('../../scripts/')
#from etl import preprocess_population

In [38]:
# Anuj population code from ETL --> should be able to just import this # ignore the hack
population = spark.read.option("header", True).csv('../../data/tables/population_data.csv')

In [39]:
from pyspark.sql.types import LongType, IntegerType
def population_preprocess(data):

    cols_to_keep = ['sa2_maincode_2016', 'sa2_name_2016', 'erp_2021']
    population_df = data.select(*cols_to_keep)

    population_df = population_df \
                    .withColumn("sa2_maincode_2016", F.col("sa2_maincode_2016").cast(LongType())) \
                    .withColumn("erp_2021", F.col('erp_2021').cast(IntegerType()))

    population_df = population_df \
                    .withColumnRenamed('sa2_name_2016', 'suburb') \
                    .withColumnRenamed('erp_2021', 'estimated_region_population_2021')

    return population_df

In [40]:
population = population_preprocess(population)

In [78]:
print("Before linking rows (population): " , population.count())
print("Before linking rows (postcode links): " , link.count())
#population.head(5)
population.orderBy(F.col('estimated_region_population_2021').desc()).limit(5)

Before linking rows (population):  2292


Before linking rows (postcode links):  5588


sa2_maincode_2016,suburb,estimated_region_population_2021
212031301,Cranbourne East,60489
213051365,Tarneit,52866
116021312,Riverstone - Mars...,48063
504031059,Ellenbrook,47743
206041122,Melbourne,47192


In [69]:
link.limit(5)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002
0800,701011002
0801,701011002


In [42]:
# investigate
print("Distinct SA2 Codes: (link)", link.select('sa2_code').distinct().count())
print("Distinct postcodes (link): ", link.select('postcode').distinct().count())
print("Distinct entries: (link)", link.distinct().count())

print("Distinct SA2 Codes: (population)", population.select('sa2_maincode_2016').distinct().count())
#print("Distinct postcodes (link): ", link.select('postcode').distinct().count())
#print("Distinct entries: (link)", link.select('postcode').distinct().count())


Distinct SA2 Codes: (link) 2222
Distinct postcodes (link):  3167
Distinct entries: (link) 5588
Distinct SA2 Codes: (population) 2292


In [54]:
population_joined = population.join(link, population['sa2_maincode_2016'] == link['sa2_code'], "left").na.drop().distinct()
#population_joined = population.join(link, population['sa2_maincode_2016'] == link['sa2_code'], "inner").distinct()

In [80]:
print(population_joined.count())
population_joined.orderBy('postcode').orderBy('postcode').limit(20)
#population_joined.where(population_joined['postcode'] == '3221') # check that some numbers look right

5492


sa2_maincode_2016,suburb,estimated_region_population_2021,postcode,sa2_code
203031050,Ocean Grove - Bar...,30630,3221,203031050
203021038,Corio - Norlane,27375,3221,203021038
203031051,Portarlington,9040,3221,203031051
203021039,Geelong,13778,3221,203021039
203011034,Bannockburn,7725,3221,203011034
203021045,Newcomb - Moolap,14938,3221,203021045
203021042,Highton,24126,3221,203021042
203021043,Lara,21289,3221,203021043
217031476,Otway,3661,3221,217031476
203021047,North Geelong - B...,15586,3221,203021047


In [57]:
print("distinct postcodes = ", population_joined.select('postcode').distinct().count())
print("distinct sa2_codes = ", population_joined.select('sa2_code').distinct().count())
#population_joined.na.drop().count()

distinct postcodes =  3165
distinct sa2_codes =  2221


In [58]:
agg_by_postcode_population = population_joined \
        .groupBy('postcode') \
        .agg(
        F.sum('estimated_region_population_2021').alias('estimated_region_population_2021_sum'),
        ) \
        .orderBy('postcode')

In [59]:
agg_by_postcode_population.limit(10)

postcode,estimated_region_population_2021_sum
0200,2161
0800,7088
0801,7088
0804,2827
0810,35007
0811,3154
0812,18442
0813,2848
0814,3719
0815,3154


Check schema types

In [60]:
transactions.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- category: string (nullable = true)



In [61]:
transactions.count()

8151372

In [66]:
# rename postcode cols before joining
agg_by_postcode_population = agg_by_postcode_population.withColumnRenamed('postcode', 'postcode_pset')
agg_by_postcode_income = agg_by_postcode_income.withColumnRenamed('postcode', 'postcode_iset')

In [67]:
# merge with transactions ->
transactions_ = transactions\
    .join(agg_by_postcode_population, transactions['postcode'] == agg_by_postcode_population['postcode_pset'], how='left')\
    .drop('postcode_pset')\
    .join(agg_by_postcode_income, transactions['postcode'] == agg_by_postcode_income['postcode_iset'], how='left')\
    .drop('postcode_iset')

In [76]:
#transactions_.orderBy(F.col('estimated_region_population_2021_sum').desc()).limit(5)
transactions_.orderBy(F.col('estimated_region_population_2021_sum').asc_nulls_last()).limit(5)
transactions_.orderBy(F.col('estimated_region_population_2021_sum').asc_nulls_first()).limit(5)

merchant_abn,consumer_id,user_id,dollar_value,order_id,order_datetime,state,postcode,gender,merchant_name,tag,revenue,rate,category,estimated_region_population_2021_sum,persons_earners_2018-19_sum,mean_earnings_2018-19_avg,sum_earnings_2018-19_sum,median_earnings_2018-19_avg,med_age_earners_2018-19_avg
15912561882,888074,15727,512.09,95a7d4b7-0588-4de...,2021-04-16,VIC,3221,Female,Nullam Nisl Insti...,shoe shops,b,3.72,retail_and_wholes...,195808,105712.0,59923.416666666664,6.519140918E9,49029.75,44.333333333333336
24852446429,922827,788,24.53,1490a482-783d-4c9...,2021-07-01,VIC,3221,Female,Erat Vitae LLP,florists supplies...,c,2.94,retail_and_wholes...,195808,105712.0,59923.416666666664,6.519140918E9,49029.75,44.333333333333336
24852446429,922827,788,34.25,9a24a9af-7946-449...,2021-06-22,VIC,3221,Female,Erat Vitae LLP,florists supplies...,c,2.94,retail_and_wholes...,195808,105712.0,59923.416666666664,6.519140918E9,49029.75,44.333333333333336
22900076341,888074,15727,39.03,5b39362d-2c82-49f...,2022-02-26,VIC,3221,Female,In Consequat LLC,computer programm...,c,2.47,info_media_and_te...,195808,105712.0,59923.416666666664,6.519140918E9,49029.75,44.333333333333336
24852446429,922827,788,24.48,af53ba5e-50ff-427...,2021-08-25,VIC,3221,Female,Erat Vitae LLP,florists supplies...,c,2.94,retail_and_wholes...,195808,105712.0,59923.416666666664,6.519140918E9,49029.75,44.333333333333336


In [73]:
transactions_.count()

8151372

In [74]:
transactions_.na.drop().count()

7688993

In [81]:
transactions_.select('postcode').distinct().count()

3165

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

In [90]:
post_merge_df_check_nulls = transactions_.drop('order_datetime')

In [93]:
def check_nulls(df):
    df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            col(c).contains('null') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c
                                )).alias(c)
                     for c in df.columns])
    return df2


In [94]:
post_merge_nulls_counts = check_nulls(post_merge_df_check_nulls)

In [95]:
post_merge_nulls_counts.limit(5)

merchant_abn,consumer_id,user_id,dollar_value,order_id,state,postcode,gender,merchant_name,tag,revenue,rate,category,estimated_region_population_2021_sum,persons_earners_2018-19_sum,mean_earnings_2018-19_avg,sum_earnings_2018-19_sum,median_earnings_2018-19_avg,med_age_earners_2018-19_avg
0,0,0,0,0,0,0,0,333642,333642,333642,333642,333642,122762,134264,134264,134264,134264,134264


In [91]:
check_transactions_null = transactions.drop('order_datetime')



In [88]:
check_res = check_nulls(check_transactions_null)

In [89]:
check_res.limit(5)

merchant_abn,consumer_id,user_id,dollar_value,order_id,state,postcode,gender,merchant_name,tag,revenue,rate,category
0,0,0,0,0,0,0,0,333642,333642,333642,333642,333642
